In [126]:
import pandas as pd

In [127]:
import os

In [128]:
%pwd

'/Users/vallirajasekar/Desktop/ML/Red_Wine_Quality/End-to-End-ML-Project'

In [129]:
os.chdir("/Users/vallirajasekar/Desktop/ML/Red_Wine_Quality/End-to-End-ML-Project")

In [130]:
%pwd

'/Users/vallirajasekar/Desktop/ML/Red_Wine_Quality/End-to-End-ML-Project'

## Creation of Entity 

In [131]:
## Create the Entity -- Return type of Function

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

## Configuration manager

In [132]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml,create_directories

In [133]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [134]:
#schema

## Data Ingestion Components

In [135]:
import os
import urllib.request as request
import zipfile
#from src.mlProject import logger
from src.mlProject.utils.common import get_size
#import logging  # Importing logging module directly
from src.mlProject.logging import *
from src.mlProject.constants import *


In [136]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


## Pipeline

In [137]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-28 23:02:15,762: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-28 23:02:15,764: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-28 23:02:15,766: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-28 23:02:15,766: INFO: common: created directory at: artifacts]
[2024-03-28 23:02:15,767: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-28 23:02:17,157: INFO: 1268487314: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 22234
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "cff2ac1407ee6cd842e8a930d18e44c9b4b704357ed9ccffdff77e8bca49a4d9"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C344:211DE0:3A1BCD:4B10C8:6605A9A0
Accept-Ranges: bytes
Date: Thu, 28

In [ ]:
data_ingestion

In [138]:
data_ingestion_config

DataIngestionConfig(root_dir='artifacts/data_ingestion', source_URL='https://github.com/vallirajasekar/Branching_tutorial/raw/main/WineQT.csv.zip', local_data_file='artifacts/data_ingestion/data.zip', unzip_dir='artifacts/data_ingestion')

In [139]:
config